Des **tests de corrélation et en Analyce de Composantes Principales (ACP)** ont été effectué afin d'aider à affiner le modèle.

In [ ]:
import pandas as pd

df = pd.read_csv("cleaned_data.csv", sep=',', on_bad_lines='skip', low_memory=False)

df = df.drop(['pnns_groups_2','energy-kj_100g','code','nutrition-score-fr_100g','product_name','created_datetime','quantity','brands','categories','categories_en','pnns_groups_1','main_category_en','ingredients_text','countries_en','nutriscore_grade','product_name_lower','brands_lower'], axis = 1)



**On regarde la heatmap de corrélation entre variable**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Calculer la matrice de corrélation
correlation_matrix = df.corr()

import pandas as pd

# Calculer la matrice de corrélation
correlation_matrix = df.corr()

# Définir un seuil de corrélation
threshold = 0.7

# Parcourir chaque colonne de la matrice de corrélation
for column in correlation_matrix.columns:
    # Obtenir les corrélations pour la colonne actuelle
    correlations = correlation_matrix[column]
    
    # Filtrer les corrélations supérieures à 0.7, en excluant la corrélation avec elle-même
    high_correlations = correlations[correlations.abs() > threshold]
    
    # Si des corrélations sont trouvées, les imprimer
    if not high_correlations.empty:
        for other_column, value in high_correlations.items():
            if other_column != column:  # Exclure la corrélation avec elle-même
                print(f"La corrélation entre la colonne '{column}' et la colonne '{other_column}' est de : {value:.2f}")
        print()  # Ligne vide pour séparer les colonnes


**On suprime la colonne fat_100g**

In [ ]:
df=df.drop('fat_100g', axis = 1)

**Analyse en composantes principales**

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


# Sélectionner les colonnes numériques
numeric_df = df.select_dtypes(include=[np.number])

# Standardiser les données
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numeric_df)

# Appliquer l'ACP
pca = PCA(n_components=5)
pca_result = pca.fit_transform(scaled_data)



In [ ]:
# Matrice de corrélation
correlation_matrix = numeric_df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True, linewidths=.5)
plt.title('Matrice de Corrélation')
plt.show()

**Convertir le résultat en DataFrame**

In [ ]:
pca_df = pd.DataFrame(data=pca_result, columns=[f'PC{i+1}' for i in range(5)])

**Visualiser les résultats pour les 2 premières composantes (PC1 vs PC2) avec Plotly**

In [ ]:
fig_2d = px.scatter(pca_df, x='PC1', y='PC2', title='ACP des données (PC1 vs PC2)',
                    labels={'PC1': 'Première Composante Principale (PC1)', 'PC2': 'Deuxième Composante Principale (PC2)'},
                    width=800, height=500)
fig_2d.update_layout(xaxis=dict(zeroline=True), yaxis=dict(zeroline=True))
fig_2d.show()

**Visualiser les résultats en 3D pour les 3 premières composantes avec Plotly**

In [ ]:
fig_3d = px.scatter_3d(pca_df, x='PC1', y='PC2', z='PC3', title='ACP des données (PC1, PC2, PC3)',
                       labels={'PC1': 'Première Composante Principale (PC1)',
                               'PC2': 'Deuxième Composante Principale (PC2)',
                               'PC3': 'Troisième Composante Principale (PC3)'})
fig_3d.update_traces(marker=dict(size=5))
fig_3d.show()

**Variance expliquée par chaque composante sous forme de graphique interactif**

In [ ]:
explained_variance = pca.explained_variance_ratio_ 

**Créer un DataFrame pour la variance expliquée**

In [ ]:
variance_df = pd.DataFrame({
    'Composante Principale': [f'PC{i+1}' for i in range(len(explained_variance))],
    'Variance Expliquée': explained_variance
})

**Variance expliquée par chaque composante**

In [ ]:
for i, variance in enumerate(explained_variance):
    print(f"Variance expliquée par la composante {i+1} : {variance:.2%}")

# Afficher la variance expliquée cumulée
cumulative_variance = explained_variance.cumsum()
print(f"\nVariance expliquée cumulée : {cumulative_variance}")

**Créer un graphique à barres interactif pour visualiser la variance expliquée**

In [ ]:
fig_variance = px.bar(variance_df, x='Composante Principale', y='Variance Expliquée',
                      title='Variance expliquée par les composantes principales')
fig_variance.show()

**Obtenir la matrice des charges et les afficher**

In [ ]:
loadings = pca.components_

# Créer un DataFrame pour afficher les charges (les contributions de chaque variable dans chaque PC)
loadings_df = pd.DataFrame(loadings.T, columns=[f'PC{i+1}' for i in range(5)], index=df.columns)
print("Matrice des charges :\n", loadings_df)

In [ ]:
!pip install prince

**Calculer la corrélation de Spearman**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

correlation_spearman = df.corr(method='spearman')

# Créer un graphique de réseau
G = nx.DiGraph()

# Ajouter des nœuds et des arêtes avec des poids (corrélations)
for i in range(len(correlation_spearman.columns)):
    for j in range(i):
        if correlation_spearman.iloc[i, j] >= 0.5:  # seuil pour corrélation positive
            G.add_edge(correlation_spearman.columns[i], correlation_spearman.columns[j],
                       weight=correlation_spearman.iloc[i, j])
        elif correlation_spearman.iloc[i, j] <= -0.5:  # seuil pour corrélation négative
            G.add_edge(correlation_spearman.columns[i], correlation_spearman.columns[j],
                       weight=correlation_spearman.iloc[i, j])

# Visualiser le graphique
pos = nx.spring_layout(G)  # Positions des nœuds
weights = nx.get_edge_attributes(G, 'weight')

# Tracer les nœuds
nx.draw_networkx_nodes(G, pos, node_size=700)

# Tracer les arêtes
edges = nx.draw_networkx_edges(G, pos, arrowstyle='-|>', arrowsize=20,
                                 edge_color=[1 if weight > 0 else -1 for weight in weights.values()],
                                 width=2, alpha=0.5)

# Ajouter des flèches et définir les couleurs
for edge in G.edges(data=True):
    if edge[2]['weight'] > 0:
        nx.draw_networkx_edges(G, pos, edgelist=[edge], edge_color='b')
    else:
        nx.draw_networkx_edges(G, pos, edgelist=[edge], edge_color='r')

# Tracer les étiquettes
nx.draw_networkx_labels(G, pos)

plt.title("Graphique de Réseau des Corrélations de Spearman")
plt.axis('off')  # Pas d'axes
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

#Calcul de la matrice de corrélation
correlation_matrix = df.corr()

#Réduction des dimensions à 3D avec ACP pour garder la structure des corrélations
pca = PCA(n_components=3)
reduced_data = pca.fit_transform(correlation_matrix)

#Initialisation du graphique en 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

#Tracé des flèches pour chaque variable
for i in range(reduced_data.shape[0]):
    ax.quiver(0, 0, 0, 
              reduced_data[i, 0], reduced_data[i, 1], reduced_data[i, 2], 
              color='b', arrow_length_ratio=0.1)
    ax.text(reduced_data[i, 0], reduced_data[i, 1], reduced_data[i, 2], 
            df.columns[i], color='r')

# Ajustement du graphique
ax.set_title("3D Correlation Vector Diagram")
ax.set_xlabel("PC1")
ax.set_ylabel("PC2")
ax.set_zlabel("PC3")
plt.show()